In [10]:
# Author: Josh Brown
# Description: Create DataFrame out of an API call to Twitter for the Freedom Convoy

# Imports needed libraries
import pandas as pd
import requests
import json

In [47]:
# Single API call
bearer_token = "AAAAAAAAAAAAAAAAAAAAAIOjYgEAAAAAUxMllZ1BahK6PEKfbLk4tisgaTU%3D3SXssYXXYHiih8HKeyGMh9XTipFNAikvo4ApWfz5ggLhbzOaog"
headers = {'Authorization':('Bearer '+ bearer_token)}

url = f'https://api.twitter.com/2/tweets/search/recent?query=Freedom%20Convoy%202022&max_results=100'
url += f'&user.fields=created_at,description,entities,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld'
url += f'&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'
url += f'&expansions=attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id'
url += f'&media.fields=duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width'
url += f'&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type'
url += f'&poll.fields=duration_minutes,end_datetime,id,options,voting_status'

response = requests.request("GET", url, headers=headers)
json_data = json.loads(response.text)
clean_data = json.dumps(json_data, indent=2)
print(clean_data)

{
  "data": [
    {
      "lang": "en",
      "reply_settings": "everyone",
      "created_at": "2022-01-29T23:15:02.000Z",
      "public_metrics": {
        "retweet_count": 72,
        "reply_count": 0,
        "like_count": 0,
        "quote_count": 0
      },
      "entities": {
        "annotations": [
          {
            "start": 46,
            "end": 51,
            "probability": 0.9992,
            "type": "Place",
            "normalized_text": "Brazil"
          }
        ],
        "mentions": [
          {
            "start": 3,
            "end": 16,
            "username": "ReutersFacts",
            "id": "1232003748886700032"
          }
        ]
      },
      "text": "RT @ReutersFacts: A clip of a truck convoy in Brazil is not recent, but dates back to May 2021 and was filmed during a rally in support of\u2026",
      "id": "1487564976814379010",
      "referenced_tweets": [
        {
          "type": "retweeted",
          "id": "1486816264135876617"
       

In [48]:
# See data configurations
for item in dict(json_data).items():
    print(item[0])
    for sub_item in item[1]:
        if type(sub_item) == dict:
            print(f'\t{sub_item.keys()}') 
        else:
            print(f'\t{sub_item}')

data
	dict_keys(['lang', 'reply_settings', 'created_at', 'public_metrics', 'entities', 'text', 'id', 'referenced_tweets', 'source', 'possibly_sensitive', 'author_id', 'conversation_id'])
	dict_keys(['lang', 'reply_settings', 'created_at', 'public_metrics', 'entities', 'text', 'id', 'source', 'possibly_sensitive', 'author_id', 'conversation_id'])
	dict_keys(['lang', 'entities', 'reply_settings', 'created_at', 'public_metrics', 'text', 'id', 'referenced_tweets', 'source', 'possibly_sensitive', 'author_id', 'conversation_id', 'attachments'])
	dict_keys(['lang', 'entities', 'reply_settings', 'created_at', 'public_metrics', 'text', 'id', 'referenced_tweets', 'source', 'context_annotations', 'possibly_sensitive', 'author_id', 'conversation_id'])
	dict_keys(['lang', 'entities', 'reply_settings', 'created_at', 'public_metrics', 'text', 'id', 'referenced_tweets', 'source', 'possibly_sensitive', 'author_id', 'conversation_id', 'attachments'])
	dict_keys(['lang', 'entities', 'reply_settings', 'cr

In [50]:
# Data Extraction
tweetSource = []
tweetLikes = []
tweetText = []
tweetRetweets = []
tweetImage = []

for tweet in json_data['data']:
    media_key = ""
    if 'attachments' in tweet:
        if 'media_keys' in tweet['attachments']:
            media_key = tweet['attachments']['media_keys'][0]

    if media_key != "":
        for media in json_data['includes']['media']:
            if media['media_key'] == media_key: 
                if media['type'] == 'photo':
                    tweetSource.append(tweet['source'])
                    tweetLikes.append(int(tweet['public_metrics']['like_count']))
                    tweetText.append(tweet['text'].strip())
                    tweetImage.append(media['url'])
                    tweetRetweets.append(int(tweet['public_metrics']['like_count']))
                    
                
            
print(tweetSource)
print(tweetLikes)
print(tweetText)
print(tweetImage)
print(tweetRetweets)

['Twitter Web App', 'Twitter for iPhone', 'Twitter for iPhone']
[0, 0, 0]
['https://t.co/vfcXEYuMu9 Impressive turnout in Vernon, BC in support of Freedom Convoy 2022.  At least 400 Canadian Patriots &amp; non-stop honks of support from drivers.  Thank you, truckers &amp; all who have stayed true to Canada, its heritage, &amp; now, its future. Voice of the West. https://t.co/krpTl6pODE', 'RT @BLNewsMedia: WE ARE LIVE \n\nhttps://t.co/wgPe1edziN https://t.co/HFWtQYQeA7', 'CANADIAN FREEDOM CONVOY 22-01-29 2022 https://t.co/T2F9jhpYsu']
['https://pbs.twimg.com/media/FKTjmTwVQAAxuYs.jpg', 'https://pbs.twimg.com/media/FKSn3z5XEAAM-AJ.jpg', 'https://pbs.twimg.com/media/FKTj9R8UcAI-30o.jpg']
[0, 0, 0]


In [51]:
# Create DataFrame
testDF = pd.DataFrame(columns=['Source', 'Likes', 'Text', 'Image', 'Retweets'])
testDF['Source'] = tweetSource
testDF['Likes'] = tweetLikes
testDF['Text'] = tweetText
testDF['Image'] = tweetImage
testDF['Retweets'] = tweetRetweets
testDF


,Source,Likes,Text,Image,Retweets
0,Twitter Web App,0,https://t.co/vfcXEYuMu9 Impressive turnout in ...,https://pbs.twimg.com/media/FKTjmTwVQAAxuYs.jpg,0
1,Twitter for iPhone,0,RT @BLNewsMedia: WE ARE LIVE \n\nhttps://t.co/...,https://pbs.twimg.com/media/FKSn3z5XEAAM-AJ.jpg,0
2,Twitter for iPhone,0,CANADIAN FREEDOM CONVOY 22-01-29 2022 https://...,https://pbs.twimg.com/media/FKTj9R8UcAI-30o.jpg,0


In [82]:
# FULL API CALL
df_size = 100
max_results = 100
img_present = 0 
next_token = ""
search_term = "Freedom%20Convoy%202022"
listTweets = []
            
while img_present < df_size:

    if next_token == "":
        url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}'
    else:
        url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&next_token={next_token}'

    url += f'&user.fields=created_at,description,entities,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld'
    url += f'&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'
    url += f'&expansions=attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id'
    url += f'&media.fields=duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width'
    url += f'&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type'
    url += f'&poll.fields=duration_minutes,end_datetime,id,options,voting_status'

    bearer_token = "AAAAAAAAAAAAAAAAAAAAAIOjYgEAAAAAUxMllZ1BahK6PEKfbLk4tisgaTU%3D3SXssYXXYHiih8HKeyGMh9XTipFNAikvo4ApWfz5ggLhbzOaog"
    headers = {'Authorization':('Bearer '+ bearer_token)}
    response = requests.request("GET", url, headers=headers)
    json_data = json.loads(response.text)

    for item in json_data['includes']['media']:
        if item['type'] == 'photo':
            for tweet in json_data['data']:
                if 'attachments' in tweet:
                    if 'media_keys' in tweet['attachments']:
                        if tweet['attachments']['media_keys'][0] == item['media_key']:
                            listTweets.append(tweet)
                            img_present += 1

    next_token = json_data['meta']['next_token']

print(img_present)

101
